# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [ ]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [ ]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

print(file_path_list[0])
print(len(file_path_list))

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [ ]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        # extracting each data row one by one and append it        
        for line in csvreader:
            # print(line)
            full_data_rows_list.append(line) 
            
# get total number of rows 
print(len(full_data_rows_list))

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [ ]:
# check the number of rows
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

# Part II. Apache Cassandra coding portion of project. 

## I will to work with the CSV file titled event_datafile_new.csv, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the **event_datafile_new.csv** after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [ ]:
# Make a connection to a Cassandra instance on machine 
from cassandra.cluster import Cluster

cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [ ]:
# Create a Keyspace 
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

#### Set Keyspace

In [ ]:
# Set KEYSPACE to the keyspace specified above
session.set_keyspace('sparkify')

### Now we need to create tables to run the following queries.

In [ ]:
# read file to pandas dataframe
file = 'event_datafile_new.csv'
df = pd.read_csv(file)
df.head(2)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

To answer this question we will need to obtain (select) the artist name, song name, and song length from out table, and we will need to filter by sessionId and itemInSession.
In CQL our query looks like:

    SELECT artist, song, length FROM artsonglength WHERE session_id = 338 AND item_in_session = 4

1. We will name our table **artsonglength**
2. Our primary key will consist of partition key sessionId, and clustering key itemInSession so that we can filter by this attributes later on.
3. The columns of our table will be: session_id, item_in_session, artist, song, length.

In [ ]:
# Query 1:  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
query = "CREATE TABLE IF NOT EXISTS artsonglength"
query = query + "(session_id int, item_in_session int,  artist text, song text, length float, PRIMARY KEY (session_id, item_in_session))"
session.execute(query)     

for i, row in df.iterrows():
    query = "INSERT INTO artsonglength (session_id, item_in_session, artist, song, length)"
    query = query + "VALUES (%s, %s, %s, %s, %s)"
    session.execute(query, (int(row[8]), int(row[3]), row[0], row[9], float(row[5])))

In [ ]:
# Do a SELECT to verify that the data have been inserted into each table
# Query 1:  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
query = "SELECT artist, song, length FROM artsonglength WHERE session_id = 338 AND item_in_session = 4"

rows = session.execute(query)

for row in rows:
    print(row.artist, row.song, row.length) 

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

To answer this question we will need to obtain (select) the artist name, song name, user name and user lastname from out table, we will need to filter by userId and sessionId, and order by itemInSession. 

In CQL our query looks like:

    SELECT artist, song,  first_name, last_name FROM usersession WHERE user_id = 10 AND session_id = 182

1. We will name our table **usersession**
2. Our primary key will consist of composite partition key userId, sessionId. The reason for this is that if we only use userId as partition key, the sessionid which belongs to the same user will be put into different nodes, which will have the performance issue when the volume of data is large.
3. Our clustering key will be itemInSession so that our results are order by it.
4. The columns of our table will be: user_id, session_id, artist, song, first_name, last_name, item_in_session.

In [ ]:
# Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
query = "CREATE TABLE IF NOT EXISTS usersession"
query = query + "(user_id int, session_id int, artist text, song text, first_name text, last_name text, item_in_session int, PRIMARY KEY ((user_id, session_id), item_in_session))"
session.execute(query)

for i, row in df.iterrows():
    query = "INSERT INTO usersession (user_id, session_id, artist, song, first_name, last_name, item_in_session)"
    query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
    session.execute(query, (int(row[10]), int(row[8]), row[0], row[9], row[1], row[4], int(row[3])))    

In [ ]:
# Do a SELECT to verify that the data have been inserted into each table
query = "SELECT artist, song,  first_name, last_name FROM usersession WHERE user_id = 10 AND session_id = 182" 

rows = session.execute(query)

for row in rows:
    print(row.artist, row.song, row.first_name, row.last_name)

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

To answer this question we will need to obtain (select) the user first name and lastname from out table, and we will need to filter by song name. As user name and lastname, in large datasets, are not unique, we will add the column userId to uniquely identify users.

In CQL our query looks like:

    SELECT first_name, last_name FROM username WHERE song = 'All Hands Against His Own'

1. We will name our table **username**
2. Our primary key will consist of partition key song, and clustering key userId. This uniquely identifies our rows.
3. The columns of our table will be: song, first_name, last_name.

In [ ]:
# Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own' session.execute("DROP TABLE IF EXISTS username")
query = "CREATE TABLE IF NOT EXISTS username"
query = query + "(song text, first_name text, last_name text, PRIMARY KEY (song, first_name))"
session.execute(query)

for i, row in df.iterrows():
    query = "INSERT INTO username (song, first_name, last_name)"
    query = query + "VALUES (%s, %s, %s)"
    session.execute(query, (row[9], row[1], row[4]))

In [ ]:
# Do a SELECT to verify that the data have been inserted into each table
query = "SELECT first_name, last_name FROM username WHERE song = 'All Hands Against His Own'"

rows = session.execute(query)

for row in rows:
    print(row.first_name, row.last_name)

### Drop the tables before closing out the sessions

In [ ]:
# Drop the table before closing out the sessions
drop_tables = ['artsonglength', 'usersession', 'username']

for i in drop_tables:
    session.execute(f"DROP TABLE IF EXISTS {i}")

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()